Start!

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c
import seaborn as sns

In [3]:
import os
import shutil
# import time

In [4]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj

# import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [5]:
# import shapely
import warnings
# from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [6]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [7]:
# import seaborn as sns

In [8]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45
cc = 1664080*u.fF/u.m

In [9]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '20 mm'
design.chips.main.size['size_y'] = '10 mm'

design.variables['cpw_width'] = '15.9 um'
design.variables['cpw_gap'] = '9 um'

design.variables['trace_width'] = '12.4 um'
cpw_pin_width = 12.4*u.um
design.variables['trace_gap'] = '7 um'
cpw_gap = 7*u.um

design.variables['pad_buffer_radius'] = '30 um'
design.variables['buffer_resolution'] = '10'
design.variables['connection_pad_buffer_radius'] = '2 um'

# design.chips['main']['material'] = 'Silicon'

gui = MetalGUI(design)

Component-specific variables

In [10]:


qb_options = dict(
pad_pocket_distance_top = '40um',
connection_pads = dict(
    a = dict(loc_W  = 0, 
            loc_H =  1, 
            pad_gap = '15um',
            pad_height = '15um',
            pad_width = '80um',
            pad_cpw_extent =  '10um',
            pocket_rise = '0um',
            cpw_extend = '0um',
            pocket_extent = '0um')))


TQ_options = dict(prime_width = design.variables['cpw_width'],
               prime_gap = design.variables['cpw_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '60um',
               coupling_space = '5um',
               open_termination=False,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False)

CPW_options = Dict(trace_width = design.variables['trace_width'],
               trace_gap  = design.variables['trace_gap'],
        total_length='5 mm',
        hfss_wire_bonds = True,
        q3d_wire_bonds = True,
        fillet='30 um',
        lead = dict(start_straight='20um', end_straight = '50um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end')), )

pin_inputs = Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end'))

trans_options = Dict(hfss_wire_bonds = True,
                     q3d_wirebonds = True,
               pin_inputs=Dict(
                 start_pin=Dict(
                     component='TQ1',
                     pin='prime_start'),
                 end_pin=Dict(
                     component='TQ2',
                     pin='prime_end')))


## Single Pad--Verify Design

### Draw the single pad transmon

In [11]:
design.delete_all_components()

In [12]:
design.overwrite_enabled = True

size = 0.25*u.mm
gap = 30*u.um
size = size.to(u.um)
pocket_width = size+2*gap


op = Dict(
    pos_x = '0',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **qb_options)

q1 = transmon(design,'Q1',options = op)

gui.rebuild()
gui.zoom_on_components(['Q1'])


In [13]:
TQ_options['down_length'] = '40 um'
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.1 mm',
                                             pos_y='0.5 mm',
                                             mirror = True,
                                             coupling_length='120 um', **TQ_options))
TQ1.options['open_termination'] = False

gui.rebuild()
gui.autoscale()



In [14]:
def Construct_CPW(pad_size, offset, extend, gapp, Lj, Cj, TQx,TQy, small, TQ_mir, buffer = 150*u.um):
    gap1 = 0.056
    gap = 30*u.um
    size = pad_size.to(u.um)
    pocket_width = size+2*gap
    design.delete_component('cpw_1')
    coupling_len = extend
    q1.options['pad_height'] = '{}'.format(size)
    q1.options['pad_width'] = '{}'.format(size)
    q1.options['pocket_width'] = '{}'.format(pocket_width)
    q1.options['connection_pads']['a']['pad_width'] = '{}'.format(coupling_len)
    q1.options['connection_pads']['a']['pad_height'] = '30um-{}'.format(gapp)
    q1.options['connection_pads']['a']['pad_gap'] = '{}'.format(gapp)
    q1.options.hfss_inductance = Lj
    q1.options.q3d_inductance =  Lj
    q1.options.hfss_capacitance = Cj
    q1.options.q3d_capacitance =  Cj



    eig_all.sim.renderer.options['Lj'] = Lj
    eig_all.sim.renderer.options['Cj'] = Cj

    eig_all.sim.setup.vars = Dict(Lj1= Lj, Cj1= Cj)    
    gui.rebuild()
    

    anchors = trans_p.anchor_CPW_round(q1,buffer, gap1, 2, small = small, last_offset = offset)
    print(anchors)
    design.delete_component('cpw_1')
    gui.rebuild()
    TQ1.options.pos_x = TQx
    TQ1.options.pos_y = TQy
    TQ1.options.mirror = TQ_mir
    gui.rebuild()
    
    pin_inputs = Dict(
                start_pin=Dict(component='Q1', pin='a'),
                end_pin=Dict(component='TQ1', pin='second_end'))

    CPW_options['pin_inputs'] = pin_inputs

    qa = RouteMixed(design, 'cpw_1', options = Dict(anchors = anchors, **CPW_options))
    length = trans_p.find_total_len(qa, q1, TQ1, count_extend=False)
    gui.rebuild()
    return qa

## Simulation of couplings

import qiskit Ansys packages

In [15]:
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis

Set the path to be optimal to save screenshots

In [16]:
original = r'C:\Users\slab\Desktop\Wendy-qiskit-code\ansys.png'
path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0406\ab'

In [17]:
all = []
components  = ['Q1','cpw_1','TQ1']
nmode = 2
datas = pd.DataFrame()

In [18]:
renderer_hfss = design.renderers.hfss
renderer_q3d = design.renderers.q3d

In [19]:
c1 = LOManalysis(design, "q3d")
c1.sim.setup.min_passes  = 10
c1.sim.setup.max_passes = 20
c1.sim.setup.freq_ghz = 5

c1.sim.renderer.options['wb_threshold'] ='72um'
c1.sim.renderer.options['x_buffer_width_mm'] = 0.5
c1.sim.renderer.options['y_buffer_width_mm'] = 0.5

In [20]:
# c1.sim.renderer.options

In [21]:
eig_all = EPRanalysis(design, 'hfss')

hfss = eig_all.sim.renderer
eig_all.sim.renderer.options['wb_threshold'] = '72um'

eig_all.sim.setup.max_passes = 30
eig_all.sim.setup.max_delta_f = 0.1
eig_all.sim.setup.n_modes = nmode
eig_all.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF')
eig_all.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_all.sim.renderer.options['y_buffer_width_mm'] = 0.5


Loop over different CPW, different coupling length, different pad size

In [22]:
guesses = pd.read_csv(r'data/educated_guess_0403.csv')

startloop

In [23]:
guesses

,Unnamed: 0,Target_qb_Freq(GHz),Sizes (um),Ljs,Ljs (nH),Buffers (um),Offsets (mm),TQx,TQy,Small,TQ_mir,CPW_freq_predict(GHz),Coupling_len(um),Coupling_len,Coupling_gap(um)
0,0,4.0,250,13nH,13,188,0.000,-0.1mm,0.5mm,False,True,6.407373,47,45um,15
1,1,4.2,235,13nH,13,184,0.000,-0.1mm,0.5mm,False,True,6.608273,40,40um,15
2,2,4.4,208,13nH,13,193,0.000,-0.1mm,0.5mm,False,True,6.800850,24,22um,15
3,3,4.6,245,10nH,10,250,0.050,0.3675mm,0.5mm,True,False,6.999753,47,45um,15
4,4,4.8,223,10nH,10,210,0.055,0.03mm,0.5mm,True,False,7.200419,30,30um,15
5,5,5.0,214,10nH,10,210,0.055,0.09mm,0.5mm,True,False,7.407967,27,30um,15
6,6,5.2,235,8nH,8,210,0.042,0.3425mm,0.46mm,True,False,7.600704,25,25um,15
7,7,5.4,225,8nH,8,210,0.065,0.3375mm,0.46mm,True,False,7.807242,23,22um,15
8,8,5.6,206,8nH,8,210,0.055,0.328mm,0.46mm,True,False,8.000022,25,30um,20
9,9,5.8,193,8nH,8,252,0.200,0.3425mm,0.46mm,True,False,8.204772,30,30um,20


In [26]:
# path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0314\loop'
gap1 = 0.056
datas = pd.DataFrame()
for k in range(10):
    dat = {}
    gap = 30*u.um
    size = guesses['Sizes (um)'].values[k]*u.um
    buffer = guesses['Buffers (um)'].values[k]*u.um
    offset = guesses['Offsets (mm)'].values[k]
    coupling_len = guesses['Coupling_len(um)'].values[k]*u.um
    coupling_gap = guesses['Coupling_gap(um)'].values[k]*u.um
    Lj = guesses['Ljs'].values[k]
    Cj = jj.find_junction_capacitance(int(Lj[:-2])*u.nH)
    
    Cj1 = str(Cj.to(u.fF).value)+' fF'
    size = size.to(u.um)
    pocket_width = size+2*gap

    TQx = guesses['TQx'].values[k]
    TQy = guesses['TQy'].values[k]
    TQ_mir = guesses['TQ_mir'].values[k]
    small = guesses['Small'].values[k]

    
    dat['pad_size(mm)'] = size.to(u.mm).value
    dat['coupling_len(um)'] = coupling_len.to(u.um).value
    dat['coupling_gap(um)'] = coupling_gap.to(u.um).value
    dat['buffer(um)'] = buffer.to(u.um).value
    dat['offset']  = offset
    dat['target_freq(GHz)'] = guesses['Target_qb_Freq(GHz)'].values[k]
    dat['target_res_freq(GHz)'] = guesses['CPW_freq_predict(GHz)'].values[k]
    dat['Cj (fF)'] = Cj.to(u.fF).value
    dat['Lj (nH)'] = guesses['Ljs (nH)'].values[k]
    dat['Small_Wrap'] = small
    dat['TQx'] = TQx
    dat['TQy'] = TQy
    dat['TQ_mir'] = TQ_mir
    
    
    qa = Construct_CPW(size, offset, coupling_len, coupling_gap, Lj, Cj1, TQx,TQy, small, TQ_mir, buffer)
    for density in ['100um', '50um', '150um']:
        options = True
        qa.options.hfss_wire_bonds = options
        qa.options.q3d_wire_bonds = options
        dat['Wirebond'] = options
        dat['Wirebond_density'] = density
        c1.sim.renderer.options['wb_threshold'] = density
        gui.rebuild()
        
        TQ1.options['open_termination'] = True
        gui.rebuild()

        c1.sim.run(components=['Q1','cpw_1','TQ1'])#, open_terminations=[('Q1', 'a')])#, ('Q1', 'bus1'), ('Q1', 'bus2')])
        c1.sim.capacitance_matrix 

        # c1.sim._get_results_from_renderer()
        c_mat1 = c1.sim.capacitance_matrix
        
        dat['C_CPW_Qb'] = c_mat1['pad_top_Q1']['a_connector_pad_Q1']
        if options:
            ground = 'g_wb'
        else:
            ground = 'ground_main_plane'
        
        dat['C_ground_Qbp'] = c_mat1['pad_top_Q1'][ground]
        dat['C_CPW_ground'] = c_mat1['a_connector_pad_Q1'][ground]
        renderer_q3d.clean_active_design()

        TQ1.options['open_termination'] = False
        gui.rebuild()

        c1.sim.run(components=['Q1','cpw_1','TQ1'])#, open_terminations=[('Q1', 'a')])#, ('Q1', 'bus1'), ('Q1', 'bus2')])
        c1.sim.capacitance_matrix 

        
        c_mat = c1.sim.capacitance_matrix
        renderer_q3d.clean_active_design()
        dat['C_Qb_g'] = c_mat['pad_top_Q1']['a_connector_pad_Q1']
        # c1.sim._get_results_from_renderer()
        
        data = pd.DataFrame(dat, index = [0])
        datas = pd.concat([datas,data], ignore_index=True)
    
        
        # count = 0
        # while count<4:
        #     try:
        #         eig_all.sim.run(name="all_freqs", components=components)#, open_terminations=[('Q1', 'a')])
        #     except:
        #         print('count')
        #         renderer_hfss.clean_active_design()
        #         count += 1
        #         if count>=3:
        #             dat['Freq'+str(i+1)+'(GHz)' ] = 0
        #             for i in range(nmode):
        #                 dat['Freq_EPR'+str(i+1)+'(MHz)' ] = 0
        #                 for j in range(i+1):
        #                     dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'  ] = 0
        #             break
        #     else:
        #         convergence = pd.read_csv('hfss_eig_f_convergence.csv')
        #         conv = convergence.dropna()
            
        #         ind = list(conv.keys())[1:]
        #         for i in range(nmode):
        #             freq = conv[ind[(i)]].values[-1]
        #             dat['Freq'+str(i+1)+'(GHz)' ] = freq
                    
        #         for i in range(nmode):
        #                 hfss.modeler._modeler.ShowWindow()
        #                 hfss.set_mode(i+1,'Setup')
        #                 hfss.plot_ansys_fields('main')
        #                 hfss.save_screenshot()
        #                 hfss.clear_fields(['main'])
                        
        #                 actual = r'C:\Users\slab\Desktop\Wendy-qiskit-code\size{:.3f}_w{:.3f}_g{:.3f}_m{}_{}_L{}_C{}_O{}.png'.format((size.value),(coupling_len.to(u.um).value),(buffer),i,offset,int(Lj[:-2]), Cj, options)
        #                 os.rename(original, actual)
        #                 fail = True
        #                 i = 0
        #                 while fail:

        #                     try:
        #                         dest = shutil.move(actual, path)
        #                     except:
        #                         i+= 1
        #                         os.rename(actual,(actual+'{}'.format(i)))
        #                         actual = (actual+'{}'.format(i))
        #                     else:
        #                         fail = False
                        
        #         #Run the EPR analysis
        #         try:
        #             del eig_all.setup.junctions['jj']
        #         except:
        #             print('saddd')
        #         eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
        #                         Lj_variable='Lj1', Cj_variable='Cj1')
        #         eig_all.setup.sweep_variable = 'Lj1'
        #         eig_all.run_epr()

        #         #Save the EPR data
        #         chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
        #         freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]
                
        #         renderer_hfss.clean_active_design()
        #         for i in range(nmode):
        #             freq = freq_EPR[i]
        #             dat['Freq_EPR'+str(i+1)+'(MHz)' ] = freq
        #             for j in range(i+1):
        #                 dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)' ] = chi[i][j]

                # data = pd.DataFrame(dat, index = [0])
                # datas = pd.concat([datas,data], ignore_index=True)
                # break


    datas.to_csv(r'data\test_airbridges_cap_only_density.csv')


02:04PM 51s INFO [delete_component]: Called delete_component cpw_1, but such a component is not in the design cache dictionary of components.


OrderedDict([(0, array([0.259, 0.259])), (1, array([ 0.259, -0.259])), (2, array([-0.259, -0.259])), (3, array([-0.259,  0.259])), (4, array([0.315, 0.315])), (5, array([ 0.315, -0.315])), (6, array([-0.315, -0.315])), (7, array([-0.315,  0.315]))])


INFO 02:04PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 02:06PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:06PM [analyze]: Analyzing setup Setup
INFO 02:09PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpceurvokg.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:09PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpkrrnn7dv.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:09PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpr1ljpxpt.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 02:09PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp7bwmeek6.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

OrderedDict([(0, array([0.2495, 0.2495])), (1, array([ 0.2495, -0.2495])), (2, array([-0.2495, -0.2495])), (3, array([-0.2495,  0.2495])), (4, array([0.3055, 0.3055])), (5, array([ 0.3055, -0.3055])), (6, array([-0.3055, -0.3055])), (7, array([-0.3055,  0.3055]))])


INFO 02:30PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 02:32PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:32PM [analyze]: Analyzing setup Setup
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp38tpmbxy.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp43z4b8z_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpdyrx_9xm.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpnp8i9g6r.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

...

OrderedDict([(0, array([0.248, 0.248])), (1, array([ 0.248, -0.248])), (2, array([-0.248, -0.248])), (3, array([-0.248,  0.248])), (4, array([0.304, 0.304])), (5, array([ 0.304, -0.249])), (6, array([ 0.388, -0.249])), (7, array([0.388, 0.304]))])


INFO 04:52PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 04:53PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:53PM [analyze]: Analyzing setup Setup
INFO 04:56PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmphkwp6rga.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:56PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpk7g7mxel.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:56PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp2xmexm8o.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 04:56PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpp17xh_jv.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

OrderedDict([(0, array([0.2625, 0.2625])), (1, array([ 0.2625, -0.2625])), (2, array([-0.2625, -0.2625])), (3, array([-0.2625,  0.2625])), (4, array([0.3185, 0.3185])), (5, array([ 0.3185, -0.1185])), (6, array([ 0.4025, -0.1185])), (7, array([0.4025, 0.3185]))])


INFO 05:09PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 05:10PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:10PM [analyze]: Analyzing setup Setup
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmptycyybia.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmplz5iergc.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpl3t942z9.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 05:13PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp59mpdyq7.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

05:28PM 09s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)

05:28PM 09s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)



In [37]:
c_mat1

,a_connector_pad_Q1,ground_main_plane,pad_top_Q1
a_connector_pad_Q1,760.32000,-748.90132,-9.89627
ground_main_plane,-748.90132,924.43389,-94.70488
pad_top_Q1,-9.89627,-94.70488,105.55264


In [33]:
data = pd.DataFrame(dat, index = [0])

In [38]:
data

,pad_size(mm),coupling_len(um),coupling_gap(um),buffer(um),offset,target_freq(GHz),target_res_freq(GHz),Cj (fF),Lj (nH),Small_Wrap,TQx,TQy,TQ_mir,Wirebond,C_CPW_Qb,C_CPW_Qbp,C_Qb_g
0,0.25,47.0,15.0,188.0,0.0,4.0,6.407373,3.392323,13,False,-0.1mm,0.5mm,True,True,-9.59122,-94.99654,-104.55984


In [35]:
datas

""


end

In [28]:
datas = pd.DataFrame()

In [27]:
datas.to_csv(r'data\test_airbridges1.csv')

In [26]:
datas

,pad_size(mm),coupling_len(um),coupling_gap(um),buffer(um),offset,target_freq(GHz),target_res_freq(GHz),Cj (fF),Lj (nH),Small_Wrap,...,Freq_EPR2(MHz)True,Chi2_1(MHz)True,Chi2_2(MHz)True,Freq1(GHz)False,Freq2(GHz)False,Freq_EPR1(MHz)False,Chi1_1(MHz)False,Freq_EPR2(MHz)False,Chi2_1(MHz)False,Chi2_2(MHz)False
0,0.25,45.0,15.0,188.0,0.0,4.0,8.807373,3.392323,13,False,...,6373.551085,0.956497,0.001336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.25,45.0,15.0,188.0,0.0,4.0,8.807373,3.392323,13,False,...,6373.551085,0.956497,0.001336,4.188845,6.374082,4008.707871,188.593167,6373.551085,0.956497,0.001336


In [25]:
q1

name:    Q1
class:   Round_TransmonPocket_Single
options: 
  'pos_x'             : '0',                          
  'pos_y'             : '0',                          
  'orientation'       : '0.0',                        
  'chip'              : 'main',                       
  'layer'             : '1',                          
  'connection_pads'   : {
       'a'                 : {
            'pad_gap'           : '15.0 umum',                  
            'pad_width'         : '45.0 um',                    
            'pad_height'        : '30um-15.0 um',               
            'pad_cpw_shift'     : '5um',                        
            'pad_cpw_extent'    : '10um',                       
            'cpw_width'         : 'trace_width',                
            'cpw_gap'           : 'trace_gap',                  
            'round_corners'     : 'True',                       
            'corner_radius'     : 'connection_pad_buffer_radius',
            'cpw_extend

end loop

In [24]:
eig_all.sim.run(name="all_freqs", components=components)

INFO 08:22AM [connect_design]: 	Opened active design
	Design:    all_freqs_hfss [Solution type: Eigenmode]
INFO 08:23AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 08:23AM [analyze]: Analyzing setup Setup
08:39AM 33s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\hfss_eig_f_convergence.csv


In [27]:
q1.options['pad_pocket_distance_top']

{}

In [29]:
q1.options

{'chip': 'main',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': 'Q1', 'pin': 'a'},
  'end_pin': {'component': 'TQ1', 'pin': 'second_end'}},
 'fillet': '30 um',
 'lead': {'start_straight': '20um',
  'end_straight': '50um',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '5 mm',
 'trace_width': '12.4 um',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'anchors': {0: array([0.259, 0.259]),
  1: array([ 0.259, -0.259]),
  2: array([-0.259, -0.259]),
  3: array([-0.259,  0.259]),
  4: array([0.315, 0.315]),
  5: array([ 0.315, -0.315]),
  6: array([-0.315, -0.315]),
  7: array([-0.315,  0.315])},
 'advanced': {'avoid_collision': 'true'},
 'step_size': '0.25mm',
 'between_anchors': {},
 'hfss_wire_bonds': True,
 'q3d_wire_bonds': True,
 'trace_gap': '7 um',
 '_actual_length': '4.563088898038469 mm',
 'pad_height': '250.0 um',
 'pad_width': '250.0 um',
 'pocket_width': '310.0 um',
 'connection_